# **CDS Project: Part 3**

*Institute of Software Security (E22)*  
*Hamburg University of Technology*  

## Learning objectives
---

- Choose an ML model architecture for vulnerability prediction
- Preprocess the dataset you created in project part 2 to fit the model you selected
- Split the dataset for cross validation
- Create the model training pipeline, train the model using the train set and optimize the model using the test set
- Create loss graphs of the learning behaviour
- Generate predictions for the validation set
- Evaluate the results using appropriate metrics (add questions: reduce overfitting? optional 5-fold cross validation)


## Materials
---

- Lecture Slides 2, 3, 5 and 6, 9.
- PyTorch Documentation: [Datasets and Data Loaders](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)
-


## Project Description

In this project, you will create, train and evaluate your own machine learning model to predict vulnerabilities in source code. In addition, you will preprocess the data to prepare it to be used in the learning pipeline.
You will be using the dataset created in part 2 in this part.

## **Task 1**

- There are several ML model architectures that you can use for vulnerabilitiy prediction such as RNN, CNN, Multilabel Perceptron (MLP), Gated Recurrent Units (GRU) and so on. Choose a suitable ML model architecture for your project based on the strenghts and weaknesses of these algorithms.

- For the model architecture, you can choose from 2 paths:
  1. Preprocess the functions into vectors and continue the learning process similar to project 1.
  For instance, the code in the dataset needs to be converted into some type of vector representation that can be processed by a machine. After selecting a suitable model, preprocess the content in your dataset accordingly. Several models are available to preprocess code data such as:
    - Code2Vec: https://code2vec.org/
    - CodeBERT: https://github.com/microsoft/CodeBERT/tree/master/UniXcoder
    - CodeT5: https://github.com/salesforce/CodeT5

  2. Choose a model architecture that is capable of handeling raw text inputs (LSTM, GRU, etc.) as an input layer. Further information on text processing using pytorch:
    - https://pytorch.org/text/stable/index.html
    - https://pytorch.org/tutorials/


In [1]:
!pip install transformers datasets --quiet


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import transformers
print(transformers.__version__)


4.53.0


In [4]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, matthews_corrcoef
from sklearn.utils.class_weight import compute_class_weight

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt



# Load the dataset directly from Google Drive
df = pd.read_csv("/content/drive/MyDrive/cyber security/labeled_dataset.csv")
df.head()

function_code  label
0  /**\n * Licensed to the Apache Software Founda...      1
1  /**\n * Licensed to the Apache Software Founda...      0
2  /*\n * Licensed to the Apache Software Foundat...      1
3  /*\n * Licensed to the Apache Software Foundat...      0
4  /**\n * Licensed to the Apache Software Founda...      1

In [5]:
def clean_code(code):
    code = str(code)

    # Remove block and inline comments
    code = re.sub(r"/\*\*.*?\*/", " ", code, flags=re.DOTALL)
    code = re.sub(r"/\*.*?\*/", " ", code, flags=re.DOTALL)
    code = re.sub(r"//.*?$", " ", code, flags=re.MULTILINE)
    code = re.sub(r"#.*?$", " ", code, flags=re.MULTILINE)

    # Remove import/package lines
    code = re.sub(r'^\s*(import|package)\s+[^\n]+', '', code, flags=re.MULTILINE)

    # Replace string and numeric literals
    code = re.sub(r'"[^"\n]*"|\'[^\']*\'', 'STR', code)
    code = re.sub(r'\b\d+\b', 'NUM', code)



    # Normalize whitespace
    code = re.sub(r'\s+', ' ', code).strip()
    return code



# Apply preprocessing
df['function_code'] = df['function_code'].astype(str).fillna('')
df['code'] = df['function_code'].apply(clean_code)

# Extra safety: strip again just in case
df['code'] = df['code'].str.strip()

# Drop rows with empty strings
df = df[df['code'].astype(bool)]

# Check output
df[['code', 'label']].head(10)


code  label
0  public class XalanXPathEvaluator implements XP...      1
1  public class XalanXPathEvaluator implements XP...      0
2  public class ChannelBufferWrapper implements A...      1
3  public class ChannelBufferWrapper implements A...      0
4  public class XalanXPathEvaluator implements XP...      1
5  public class XalanXPathEvaluator implements XP...      0
6  public final class SimpleString implements Cha...      1
7  public final class SimpleString implements Cha...      0
8  public class ArtemisTest extends CliTestBase {...      1
9  public class ArtemisTest extends CliTestBase {...      0

In [6]:
print(df['function_code'].iloc[0])
print(df['code'].iloc[0])

/**
 * Licensed to the Apache Software Foundation (ASF) under one or more
 * contributor license agreements.  See the NOTICE file distributed with
 * this work for additional information regarding copyright ownership.
 * The ASF licenses this file to You under the Apache License, Version 2.0
 * (the "License"); you may not use this file except in compliance with
 * the License.  You may obtain a copy of the License at
 *
 *      http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

package org.apache.activemq.apollo.filter;

import org.apache.xpath.CachedXPathAPI;
import org.apache.xpath.objects.XObject;
import org.w3c.dom.Document;
import org.w3c.dom.traversal.NodeIterator

In [7]:
subset_df = df.sample(n=1000, random_state=42)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

## Task 2

- Split your dataset appropriately into train, test, and validation set and justify your split.  

In [9]:
# TODO: Load the dataset

# TODO: Split the dataset into train, test and validation set

In [10]:
# Prepare input texts and labels
texts = subset_df['code'].tolist()
labels = subset_df['label'].tolist()
train_texts, temp_texts, train_labels, temp_labels = train_test_split(texts, labels, test_size=0.3, stratify=labels, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.5, stratify=temp_labels, random_state=42)


## Task 3
- Create a preprocessing and training/test pipeline. This can also be done in separate steps.
- Train the model on a small dataset (e.g.: 1000 samples depending on the computational capacity) and make sure everything is working properly
- Show a graph showing the changes in loss function druring the training epochs
- Select proper metrics to evaluate the performance of the model
- If everything works well, then change the model parameters (e.g: number of neurons, layer depth) and regenerate the results. Comapare these results with your previous results and elaborate your findings.

In [11]:
# TODO: Implement your ML pipeline and perform evaluation

In [12]:
class CodeBERTSequenceDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.labels = labels
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        encoded = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoded['input_ids'].squeeze(),
            'attention_mask': encoded['attention_mask'].squeeze(),
            'label': torch.tensor(self.labels[idx], dtype=torch.long)
        }
train_dataset = CodeBERTSequenceDataset(train_texts, train_labels, tokenizer)
val_dataset = CodeBERTSequenceDataset(val_texts, val_labels, tokenizer)
test_dataset = CodeBERTSequenceDataset(test_texts, test_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

In [13]:
class CodeBERTLSTMClassifier(nn.Module):
    def __init__(self, hidden_dim=128, lstm_layers=2, dropout=0.3):
        super().__init__()
        self.codebert = AutoModel.from_pretrained("microsoft/codebert-base")
        self.lstm = nn.LSTM(input_size=768, hidden_size=hidden_dim,
                            num_layers=lstm_layers, batch_first=True,
                            bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(hidden_dim * 2, 2)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            outputs = self.codebert(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state

        lstm_out, _ = self.lstm(embeddings)
        pooled = torch.mean(lstm_out, dim=1)
        logits = self.classifier(self.dropout(pooled))
        return logits

model = CodeBERTLSTMClassifier().to(device)

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
loss_fn = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
loss_history = []
epochs = 10

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        logits = model(input_ids, attention_mask)
        loss = loss_fn(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    loss_history.append(avg_loss)
    print(f"Epoch {epoch+1} | Avg Loss: {avg_loss:.4f}")


Epoch 1 | Avg Loss: 0.7011


In [ ]:
plt.plot(loss_history)
plt.xlabel("Epoch")
plt.ylabel("Training Loss")
plt.title("Loss over Epochs")
plt.show()

In [ ]:
model.eval()
preds, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        logits = model(input_ids, attention_mask)
        predictions = torch.argmax(logits, dim=1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

print(classification_report(true_labels, preds))
print("MCC:", matthews_corrcoef(true_labels, preds))
